# MLVOT: Pratical Work 2

**Objective**: Develop a simple IoU-based tracker without using image information. Extend the algorithm to
handle multiple object tracking simultaneously.

In [15]:
DATA_DIR = "ADL-Rundle-6"
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from tqdm import tqdm
import argparse
import imageio
import itertools
from src.iou import BoundingBox, intersection_box, iou

In [23]:
BOUNDING_BOX_DIR = "ADL-Rundle-6/bounding_boxes"
IMG_DIR = "ADL-Rundle-6/img1"
IMG_FILE_LIST = sorted(os.listdir(IMG_DIR))

## 1 
Load detections (det) stored in a MOT-challenge like formatted text file. Each line represents one object
instance and contains 10 values (fieldNames = [\<frame\>, <id>, <bb_left>, <bb_top>, <bb_width>,
<bb_height>, <conf>, <x>, <y>, <z>]


In [24]:
det_file = (
    "ADL-Rundle-6/det/det.txt"
    if os.path.exists("ADL-Rundle-6/det/clean_det.csv")
    else "ADL-Rundle-6/det/det.txt"
)


def load_det_file(det_file):
    if not os.path.exists("ADL-Rundle-6/det/clean_det.csv"):
        det_df = pd.read_csv(det_file, sep=",", header=None)
        det_df.columns = [
            "frame",
            "id",
            "bb_left",
            "bb_top",
            "bb_width",
            "bb_height",
            "conf",
            "x",
            "y",
            "z",
        ]
        det_df.to_csv("ADL-Rundle-6/det/clean_det.csv", index=False)
    else:
        det_df = pd.read_csv("ADL-Rundle-6/det/clean_det.csv", sep=",", header=0)
    return det_df


det_df = load_det_file(det_file)

## 2 Implement IoU for tracking

- Compute similarity score using the Jaccard index (intersection-over-union) for each pair of
bounding boxes

Jacard index between 2 bounding boxes is implemented in [this script](./src/iou.py)

In [28]:
def iou_tracking(
    det_df,
    img_file_list=IMG_FILE_LIST,
    threshold=0.5,
    save_gif=False,
    csv_file="ADL-Rundle-6/det/restults.csv",
):
    cur_id = 0
    for n_frame, img_file in tqdm(enumerate(img_file_list, start=1)):
        frame_data = det_df[det_df["frame"] == n_frame]
        next_frame_data = det_df[det_df["frame"] == n_frame + 1]
        for i, row1 in enumerate(frame_data.index):
            for j, row2 in enumerate(next_frame_data.index):
                bb1 = BoundingBox(
                    frame_data["bb_left"][row1],
                    frame_data["bb_top"][row1],
                    frame_data["bb_width"][row1],
                    frame_data["bb_height"][row1],
                )
                bb2 = BoundingBox(
                    next_frame_data["bb_left"][row2],
                    next_frame_data["bb_top"][row2],
                    next_frame_data["bb_width"][row2],
                    next_frame_data["bb_height"][row2],
                )
                iou_score = iou(bb1, bb2)

                if det_df.loc[row1, "id"] == -1:
                    det_df.loc[row1, "id"] = cur_id
                    cur_id += 1
                if iou_score >= threshold:
                    det_df.loc[row2, "id"] = det_df.loc[row1, "id"]

    det_df.to_csv(csv_file, index=False)


iou_tracking(
    det_df,
    csv_file="ADL-Rundle-6/det/restults.csv",
    threshold=0.5,
    img_file_list=IMG_FILE_LIST,
)

525it [00:01, 320.93it/s]


In [29]:
def update_gif(opencv_img, row1, bb1, img_file):
    cv2.rectangle(
        opencv_img,
        (int(bb1.bb_left), int(bb1.bb_top)),
        (
            int(bb1.bb_left + bb1.bb_width),
            int(bb1.bb_top + bb1.bb_height),
        ),
        (0, 0, 255),
        2,
    )
    cv2.putText(
        opencv_img,
        str(det_df.loc[row1, "id"]),
        (int(bb1.bb_left), int(bb1.bb_top)),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 0, 255),
        2,
        cv2.LINE_AA,
    )
    cv2.imwrite(
        os.path.join(BOUNDING_BOX_DIR, img_file),
        opencv_img,
    )

In [30]:
def generate_gif(
    result_csv="ADL-Rundle-6/det/restults.csv",
    img_file_list=IMG_FILE_LIST,
    gif_file="ADL-Rundle-6/bounding_boxes.gif",
    nb_frames=10,
):
    df = pd.read_csv(result_csv, sep=",", header=0)
    img_file_list = img_file_list[:nb_frames]
    for n_frame, img_file in tqdm(enumerate(img_file_list, start=1)):
        res_df = df[df["frame"] == n_frame]
        opencv_img = cv2.imread(os.path.join(IMG_DIR, img_file))
        for row1 in res_df.index:
            bb1 = BoundingBox(
                res_df["bb_left"][row1],
                res_df["bb_top"][row1],
                res_df["bb_width"][row1],
                res_df["bb_height"][row1],
            )
            update_gif(opencv_img, row1, bb1, img_file)
    images = []
    print("Generating gif...")
    bounded_box_files = sorted(os.listdir(BOUNDING_BOX_DIR))[:nb_frames]
    for filename in tqdm(bounded_box_files):
        images.append(imageio.imread(os.path.join(BOUNDING_BOX_DIR, filename)))
    imageio.mimsave(gif_file, images, duration=0.5)
    print("Gif saved at {}".format(gif_file))


generate_gif(
    result_csv="ADL-Rundle-6/det/restults.csv",
    img_file_list=IMG_FILE_LIST,
    gif_file="ADL-Rundle-6/bounding_boxes.gif",
    nb_frames=10,
)

10it [00:01,  7.96it/s]


Generating gif...


  0%|          | 0/10 [00:00<?, ?it/s]/tmp/ipykernel_30561/2065052759.py:24: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(os.path.join(BOUNDING_BOX_DIR, filename)))
100%|██████████| 10/10 [00:00<00:00, 56.06it/s]


Gif saved at ADL-Rundle-6/bounding_boxes.gif


## 3 Detection Association
Associate the detections to tracks in a greedy manner using IoU/ threshold sigma_iou. A track gets the
detection with the highest intersection-over-union to its last known object position (i.e. the previous
detection of the track) assigned.

## 4 Track management
 Each object can be assigned to only one trajectory (ID)
 Create and update lists for matches, unmatched detections and unmatched tracks
 Matches: IoU ≥ sigma_iou -> matched track
 Unmatched tracks -> delete track
 Unmatched detection -> create new tracks